In [1]:
from nltk.tokenize import WordPunctTokenizer
import numpy as np
import pandas as pd
import re
import numpy as np
import pandas as pd 

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split

from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM
from keras.datasets import imdb
from keras import layers

from keras.utils.np_utils import to_categorical

import warnings
warnings.filterwarnings('ignore')
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

Using TensorFlow backend.
c:\users\hp\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\users\hp\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\users\hp\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
c:

In [6]:
path="C:/Users/Apar/Desktop/india-news-headlines.csv"
df=pd.read_csv(path)
df.head()

,publish_date,headline_category,headline_text
0,20010101,sports.wwe,win over cena satisfying but defeating underta...
1,20010102,bollywood,Raju Chacha
2,20010102,unknown,Status quo will not be disturbed at Ayodhya; s...
3,20010102,unknown,Fissures in Hurriyat over Pak visit
4,20010102,unknown,America's unwanted heading for India?


In [7]:
df.shape

(2969922, 3)

In [8]:
df=df[df.headline_category!='unknown']

In [9]:
df=df[:20000]

In [10]:
df.head()

,publish_date,headline_category,headline_text
0,20010101,sports.wwe,win over cena satisfying but defeating underta...
1,20010102,bollywood,Raju Chacha
87,20010103,entertainment.hindi.bollywood.news,'Devdas': Jinxed?
128,20010104,business.india-business,Car dealers caught in Bihar sales tax ruling
129,20010104,city.bengaluru,He's not so inscrutable


In [12]:
df.headline_category.nunique() 

40

In [13]:
k=list(df.headline_category.unique())

In [14]:
for i in k:
    print(i)

sports.wwe
bollywood
entertainment.hindi.bollywood.news
business.india-business
city.bengaluru
city.delhi
city.patna
hollywood
india
sports.cricket.news
sports.football
only-in-america
city.ahmedabad
jugular-vein
swaminomics
city.thiruvananthapuram
city.chandigarh
city.mumbai
city.pune
home.science
city.kolkata
city.lucknow
city.hyderabad
cricket
business.international-business
removed
entertainment.english.hollywood.news
off-the-cuff
home.sunday-times.deep-focus
bombay-times
india-in-lanka
home.sunday-times.all-that-matters
talking-terms
home.education
pune-times
calcutta-times
hyderabad-times
bangalore-times
delhi-times
lucknow-times


In [15]:
for i in range (df.shape[0]):
   # df.loc[1].at['B']
    j=df['headline_category'].iloc[i].split(".")
    df['headline_category'].iloc[i]=j[0]

In [16]:
k=list(df.headline_category.unique())

In [17]:
c=0
for i in k:
    print(i)

sports
bollywood
entertainment
business
city
hollywood
india
only-in-america
jugular-vein
swaminomics
home
cricket
removed
off-the-cuff
bombay-times
india-in-lanka
talking-terms
pune-times
calcutta-times
hyderabad-times
bangalore-times
delhi-times
lucknow-times


In [18]:
maxi=0
for i in range (df.shape[0]):
   # df.loc[1].at['B']
    j=len(df['headline_text'].iloc[i].split(" "))
    if maxi<j:
        maxi=j
maxi

17

In [19]:
df['headline_text'] = df['headline_text'].apply(lambda x: x.lower())
df['headline_text'] = df['headline_text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))
    
max_fatures = 14
tokenizer = Tokenizer(nb_words=max_fatures, split=' ')
tokenizer.fit_on_texts(df['headline_text'].values)
x = tokenizer.texts_to_sequences(df['headline_text'].values)
x = pad_sequences(x)
print(x.shape)
print(x)

(20000, 7)
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 1 5]
 [0 0 0 ... 0 1 1]
 [0 0 0 ... 0 0 0]]


In [20]:
input_dim = x.shape[1]
y = pd.get_dummies(df['headline_category']).values
y

array([[0, 0, 0, ..., 1, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint8)

In [21]:
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size = 0.3, random_state = 42)
print('Shape of training samples:',x_train.shape,y_train.shape)
print('Shape of testing samples:',x_test.shape,y_test.shape)

Shape of training samples: (14000, 7) (14000, 23)
Shape of testing samples: (6000, 7) (6000, 23)


In [30]:
model = Sequential()
model.add(Dense(32,input_dim=input_dim, activation='relu'))
model.add(Dense(128))
model.add(Dense(y_train.shape[1], activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_9 (Dense)              (None, 32)                256       
_________________________________________________________________
dense_10 (Dense)             (None, 128)               4224      
_________________________________________________________________
dense_11 (Dense)             (None, 23)                2967      
Total params: 7,447
Trainable params: 7,447
Non-trainable params: 0
_________________________________________________________________
None


In [31]:
history = model.fit(x_train, y_train,
                    epochs=10,
                     verbose=1,
                    validation_data=(x_test, y_test))

Train on 14000 samples, validate on 6000 samples
Epoch 1/10
14000/14000 [==============================] - 1s 48us/step - loss: 1.5821 - acc: 0.6363 - val_loss: 1.4073 - val_acc: 0.6398
Epoch 2/10
14000/14000 [==============================] - 0s 29us/step - loss: 1.4074 - acc: 0.6412 - val_loss: 1.3989 - val_acc: 0.6400
Epoch 3/10
14000/14000 [==============================] - 0s 29us/step - loss: 1.3993 - acc: 0.6414 - val_loss: 1.3955 - val_acc: 0.6400
Epoch 4/10
14000/14000 [==============================] - 0s 29us/step - loss: 1.3940 - acc: 0.6412 - val_loss: 1.3832 - val_acc: 0.6402
Epoch 5/10
14000/14000 [==============================] - 0s 29us/step - loss: 1.3887 - acc: 0.6413 - val_loss: 1.3862 - val_acc: 0.6402
Epoch 6/10
14000/14000 [==============================] - 0s 33us/step - loss: 1.3840 - acc: 0.6414 - val_loss: 1.3858 - val_acc: 0.6402
Epoch 7/10
14000/14000 [==============================] - 0s 31us/step - loss: 1.3823 - acc: 0.6411 - val_loss: 1.3811 - val_acc:

In [33]:
vocab_size = len(tokenizer.word_index) + 1
vocab_size

18679

In [35]:
model = Sequential()
model.add(Embedding(vocab_size, 100 ,input_length = x.shape[1], dropout=0.2))
model.add(layers.Flatten())
model.add(Dense(128))
model.add(Dense(32))
model.add(Dense(y_train.shape[1], activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 7, 100)            1867900   
_________________________________________________________________
flatten_1 (Flatten)          (None, 700)               0         
_________________________________________________________________
dense_12 (Dense)             (None, 128)               89728     
_________________________________________________________________
dense_13 (Dense)             (None, 32)                4128      
_________________________________________________________________
dense_14 (Dense)             (None, 23)                759       
Total params: 1,962,515
Trainable params: 1,962,515
Non-trainable params: 0
_________________________________________________________________
None


In [36]:
batch_size = 32
history = model.fit(x_train, y_train, epochs = 10, batch_size=batch_size, verbose = 2, validation_data=(x_test, y_test))

Train on 14000 samples, validate on 6000 samples
Epoch 1/10
 - 12s - loss: 1.4155 - acc: 0.6401 - val_loss: 1.3716 - val_acc: 0.6402
Epoch 2/10
 - 11s - loss: 1.3661 - acc: 0.6412 - val_loss: 1.3909 - val_acc: 0.6402
Epoch 3/10
 - 12s - loss: 1.3639 - acc: 0.6414 - val_loss: 1.3614 - val_acc: 0.6402
Epoch 4/10
 - 12s - loss: 1.3590 - acc: 0.6412 - val_loss: 1.3676 - val_acc: 0.6402
Epoch 5/10
 - 12s - loss: 1.3571 - acc: 0.6411 - val_loss: 1.3654 - val_acc: 0.6402
Epoch 6/10
 - 12s - loss: 1.3557 - acc: 0.6411 - val_loss: 1.3606 - val_acc: 0.6402
Epoch 7/10
 - 12s - loss: 1.3538 - acc: 0.6414 - val_loss: 1.3614 - val_acc: 0.6402
Epoch 8/10
 - 12s - loss: 1.3525 - acc: 0.6415 - val_loss: 1.3585 - val_acc: 0.6397
Epoch 9/10
 - 12s - loss: 1.3500 - acc: 0.6416 - val_loss: 1.3701 - val_acc: 0.6400
Epoch 10/10
 - 12s - loss: 1.3518 - acc: 0.6414 - val_loss: 1.3802 - val_acc: 0.6402


In [43]:

def create_embedding_matrix(filepath, word_index, embedding_dim):
    vocab_size = len(word_index) + 1  # Adding again 1 because of reserved 0 index
    embedding_matrix = np.zeros((vocab_size, embedding_dim))

    with open(filepath,encoding='UTF-8') as f:
        for line in f:
            word, *vector = line.split()
            if word in word_index:
                idx = word_index[word] 
                embedding_matrix[idx] = np.array(
                    vector, dtype=np.float32)[:embedding_dim]

    return embedding_matrix

In [44]:
embedding_dim = 100
embedding_matrix = create_embedding_matrix(
    'C:/Srishtee/Btech/Sem6/NLP/Assignment/word embeddings/glove.6B.100d.txt',
    tokenizer.word_index, embedding_dim)

In [46]:
model = Sequential()
model.add(layers.Embedding(vocab_size, embedding_dim, 
                           weights=[embedding_matrix], 
                           input_length=x.shape[1], 
                           trainable=False))
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(y_train.shape[1], activation='softmax'))
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 7, 100)            1867900   
_________________________________________________________________
flatten_2 (Flatten)          (None, 700)               0         
_________________________________________________________________
dense_15 (Dense)             (None, 128)               89728     
_________________________________________________________________
dense_16 (Dense)             (None, 23)                2967      
Total params: 1,960,595
Trainable params: 92,695
Non-trainable params: 1,867,900
_________________________________________________________________


In [47]:
history = model.fit(x_train, y_train,
                    epochs=10,
                    verbose=1,
                    validation_data=(x_test, y_test))

Train on 14000 samples, validate on 6000 samples
Epoch 1/10
14000/14000 [==============================] - 1s 105us/step - loss: 0.1011 - acc: 0.9660 - val_loss: 0.0902 - val_acc: 0.9688
Epoch 2/10
14000/14000 [==============================] - 1s 80us/step - loss: 0.0901 - acc: 0.9688 - val_loss: 0.0903 - val_acc: 0.9687
Epoch 3/10
14000/14000 [==============================] - 1s 73us/step - loss: 0.0898 - acc: 0.9687 - val_loss: 0.0902 - val_acc: 0.9686
Epoch 4/10
14000/14000 [==============================] - 1s 80us/step - loss: 0.0894 - acc: 0.9689 - val_loss: 0.0902 - val_acc: 0.9686
Epoch 5/10
14000/14000 [==============================] - 1s 68us/step - loss: 0.0892 - acc: 0.9689 - val_loss: 0.0900 - val_acc: 0.9686
Epoch 6/10
14000/14000 [==============================] - 1s 67us/step - loss: 0.0890 - acc: 0.9689 - val_loss: 0.0903 - val_acc: 0.9687
Epoch 7/10
14000/14000 [==============================] - 1s 67us/step - loss: 0.0888 - acc: 0.9689 - val_loss: 0.0904 - val_acc

In [66]:
tokenizer.word_index

{'to': 1,
 'in': 2,
 'for': 3,
 'of': 4,
 'on': 5,
 'the': 6,
 'a': 7,
 'govt': 8,
 'up': 9,
 'at': 10,
 'with': 11,
 'from': 12,
 'be': 13,
 'is': 14,
 'police': 15,
 'over': 16,
 'against': 17,
 'no': 18,
 'and': 19,
 'not': 20,
 'new': 21,
 'rs': 22,
 'india': 23,
 'by': 24,
 'city': 25,
 'killed': 26,
 'case': 27,
 'bjp': 28,
 'as': 29,
 'out': 30,
 'will': 31,
 'it': 32,
 'more': 33,
 'meet': 34,
 'power': 35,
 'may': 36,
 'hc': 37,
 'centre': 38,
 'into': 39,
 'cm': 40,
 'state': 41,
 'us': 42,
 'held': 43,
 'get': 44,
 'students': 45,
 'its': 46,
 'cong': 47,
 'says': 48,
 'probe': 49,
 'water': 50,
 'gets': 51,
 'all': 52,
 '3': 53,
 'two': 54,
 'set': 55,
 'chief': 56,
 'soon': 57,
 'has': 58,
 'panel': 59,
 'life': 60,
 'strike': 61,
 'back': 62,
 'cr': 63,
 'minister': 64,
 'indian': 65,
 '2': 66,
 'seeks': 67,
 'sc': 68,
 'take': 69,
 'are': 70,
 'today': 71,
 'man': 72,
 'go': 73,
 'court': 74,
 'issue': 75,
 'after': 76,
 'pm': 77,
 'plan': 78,
 'an': 79,
 'have': 80,
 'a

In [82]:
from googletrans import Translator
translator = Translator()
translated = translator.translate('आयकर छूट खत्म करने की कोई समय सीमा नहीं, इन विकल्पों पर ले सकते हैं छूट का लाभ').text.split(" ")
print(translated)

['No', 'deadline', 'for', 'tax', 'exempt,', 'can', 'take', 'on', 'these', 'options', 'advantage', 'of', 'discounts']


In [86]:
f=[]
for i in translated:
    i=i.lower()
    if i in tokenizer.word_index.keys():
        f.append(i)
print(f)

['no', 'deadline', 'for', 'tax', 'can', 'take', 'on', 'these', 'options', 'advantage', 'of']


In [84]:
f=" ".join(f)
f

'no deadline for tax can take on these options advantage of'

In [91]:
x_test1 = tokenizer.texts_to_sequences(f)
x_test1= pad_sequences(x_test1,7)
x_test1.shape

(11, 7)

In [93]:
t=model.predict(x_test1)